download file from here https://www.ipcinfo.org/ipc-country-analysis/population-tracking-tool/en/ file downloaded manually

code adjusted for linux and for higher number of files 

In [3]:
import os
import re
from datetime import datetime, timedelta
import yt_dlp
import requests
from bs4 import BeautifulSoup
import time
import tempfile
import shutil

def validate_soundcloud_url(url):
    """Validate if the provided URL is a SoundCloud URL."""
    pattern = r'^https?://(?:www\.)?soundcloud\.com/[\w-]+/[\w-]+'
    return bool(re.match(pattern, url))

def download_soundcloud_audio(url, output_dir="downloads", max_retries=3):
    """Download audio from SoundCloud URL with a temp directory approach to prevent partial files."""
    if not validate_soundcloud_url(url):
        print(f"Invalid SoundCloud URL: {url}")
        return False

    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Create a temporary directory for downloads
    with tempfile.TemporaryDirectory() as temp_dir:
        # Configure yt-dlp options with explicit ffmpeg path and no metadata
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': os.path.join(temp_dir, '%(title)s.%(ext)s'),
            'noplaylist': True,
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'ffmpeg_location': '/usr/bin/ffmpeg',
            'quiet': False,
            'verbose': False,
            'retries': 10,
            'fragment_retries': 10,
            'skip_unavailable_fragments': False,
            'abort_on_unavailable_fragment': True,
            # Skip metadata download
            'writethumbnail': False,
            'writeinfojson': False,
            'writesubtitles': False,
            'writeautomaticsub': False,
            'writedescription': False,
            'no_write_playlist_metafiles': True,
            'add_metadata': False,
            'embed_thumbnail': False,
            'embed_metadata': False
        }

        # Download with retries
        for attempt in range(max_retries):
            try:
                with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                    info = ydl.extract_info(url, download=True)
                    filename = ydl.prepare_filename(info)
                    base, _ = os.path.splitext(filename)
                    mp3_file = f"{base}.mp3"
                    
                    # Get the path to the MP3 file in temp directory
                    temp_mp3_path = os.path.join(temp_dir, os.path.basename(mp3_file))
                    
                    # Verify the file exists and isn't zero bytes
                    if os.path.exists(temp_mp3_path) and os.path.getsize(temp_mp3_path) > 0:
                        # Move the completed file to the output directory
                        dest_path = os.path.join(output_dir, os.path.basename(temp_mp3_path))
                        shutil.move(temp_mp3_path, dest_path)
                        print(f"Downloaded: {dest_path}")
                        return dest_path
                    else:
                        raise Exception(f"File does not exist or is empty: {temp_mp3_path}")
                        
            except Exception as e:
                print(f"Error downloading {url} (Attempt {attempt+1}/{max_retries}): {str(e)}")
                time.sleep(5 * (attempt + 1))
    
    print(f"Failed to download {url} after {max_retries} attempts")
    return False

def get_soundcloud_urls_by_date_range(profile_url, start_date, end_date):
    """
    Get SoundCloud URLs within a specific date range.
    
    Args:
        profile_url: The SoundCloud profile URL (e.g., "https://soundcloud.com/radio-ergo")
        start_date: Start date (datetime object)
        end_date: End date (datetime object)
        
    Returns:
        A list of SoundCloud URLs within the specified date range
    """
    if not profile_url.endswith('/'):
        profile_url = profile_url + '/'

    try:
        response = requests.get(profile_url, headers={'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()
    except Exception as e:
        print(f"Error fetching profile page: {str(e)}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all track links
    track_urls = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.startswith('/') and profile_url.split('/')[3] in href and '/sets/' not in href:
            full_url = f"https://soundcloud.com{href}"
            if validate_soundcloud_url(full_url):
                track_urls.append(full_url)

    # Filter URLs by date
    urls_in_range = []

    # Month names mapping
    month_names = {
        'january': 1, 'february': 2, 'march': 3, 'april': 4, 'may': 5, 'june': 6,
        'july': 7, 'august': 8, 'september': 9, 'october': 10, 'november': 11, 'december': 12,
        'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'jun': 6, 'jul': 7,
        'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
    }

    for url in track_urls:
        date_match = re.search(r'(\d{1,2})-([a-z]+)-(\d{4})', url.split('/')[-1], re.IGNORECASE)

        if date_match:
            day, month_str, year = date_match.groups()
            month = month_names.get(month_str.lower())

            if month:
                try:
                    track_date = datetime(int(year), month, int(day))
                    if start_date <= track_date <= end_date:
                        urls_in_range.append(url)
                        print(f"Found matching URL: {url}")
                except ValueError:
                    continue
            else:
                print(f"Could not parse month: {month_str} in URL: {url}")
        else:
            print(f"URL does not match date pattern: {url}")

    return urls_in_range

def get_existing_files(output_dir):
    """Get a list of existing complete MP3 files."""
    if not os.path.exists(output_dir):
        return []
    
    mp3_files = [f for f in os.listdir(output_dir) if f.endswith('.mp3') and not f.endswith('.part')]
    
    valid_files = []
    for file in mp3_files:
        file_path = os.path.join(output_dir, file)
        if os.path.getsize(file_path) > 0:
            valid_files.append(file)
    
    return valid_files

def download_by_date_range(profile_url, start_date_str, end_date_str, output_dir=None, save_to_drive=True):
    """
    Download SoundCloud tracks within a date range from earliest to latest date.
    
    Args:
        profile_url: The SoundCloud profile URL
        start_date_str: Start date in format 'YYYY-MM-DD'
        end_date_str: End date in format 'YYYY-MM-DD'
        output_dir: Output directory (default: None, which uses date range as folder name)
        save_to_drive: Whether to save files to Google Drive (default: True)
    """
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_str, '%Y-%m-%d')

    if not output_dir:
        output_dir = f"soundcloud_{start_date_str}_to_{end_date_str}"

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    print(f"Searching for tracks from {start_date_str} to {end_date_str}...")

    existing_files = get_existing_files(output_dir)
    if existing_files:
        print(f"Found {len(existing_files)} existing complete downloads")
        
    # Option 1: Try to find using the profile page
    urls = get_soundcloud_urls_by_date_range(profile_url, start_date, end_date)

    # Option 2: Generate URLs for all days in the range
    manual_urls = []
    current_date = start_date
    while current_date <= end_date:
        day = current_date.day
        month = current_date.strftime('%b').upper()
        year = current_date.year

        expected_filename = f"IDAACADDA {day:02d}-{month}-{year}.mp3"
        alt_expected_filename = f"IDAACADDA {day}-{month}-{year}.mp3"
        
        if expected_filename not in existing_files and alt_expected_filename not in existing_files:
            potential_urls = [
                f"https://soundcloud.com/radio-ergo/idaacadda-{day:02d}-{month.lower()}-{year}",
                f"https://soundcloud.com/radio-ergo/idaacadda-{day}-{month.lower()}-{year}"
            ]

            for url in potential_urls:
                try:
                    response = requests.head(url)
                    if response.status_code == 200:
                        # Store URL with its date for sorting
                        manual_urls.append((current_date, url))
                        print(f"Manually found URL: {url} for date {current_date.strftime('%Y-%m-%d')}")
                        break
                except:
                    pass

        current_date += timedelta(days=1)

    # Create a list of tuples with (date, url) for all URLs
    dated_urls = []
    
    # Process URLs from profile page - extract dates from them
    for url in urls:
        date_match = re.search(r'(\d{1,2})-([a-z]+)-(\d{4})', url.split('/')[-1], re.IGNORECASE)
        if date_match:
            day, month_str, year = date_match.groups()
            month_names = {
                'january': 1, 'february': 2, 'march': 3, 'april': 4, 'may': 5, 'june': 6,
                'july': 7, 'august': 8, 'september': 9, 'october': 10, 'november': 11, 'december': 12,
                'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'jun': 6, 'jul': 7,
                'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
            }
            month = month_names.get(month_str.lower())
            if month:
                try:
                    track_date = datetime(int(year), month, int(day))
                    dated_urls.append((track_date, url))
                except ValueError:
                    # If we can't parse the date, add it with a default date to the end
                    dated_urls.append((end_date, url))
            else:
                # If month parsing fails, add it with a default date to the end
                dated_urls.append((end_date, url))
        else:
            # If no date in URL, add it with a default date to the end
            dated_urls.append((end_date, url))
    
    # Combine both lists of dated URLs
    all_dated_urls = dated_urls + manual_urls
    
    # Remove duplicates while preserving order
    seen_urls = set()
    unique_dated_urls = []
    for date, url in all_dated_urls:
        if url not in seen_urls:
            seen_urls.add(url)
            unique_dated_urls.append((date, url))
    
    # Sort by date (earliest first)
    unique_dated_urls.sort(key=lambda x: x[0])
    
    # Extract just the URLs in order
    all_urls = [url for _, url in unique_dated_urls]

    if not all_urls:
        print("No new tracks found to download.")
        return [], []

    print(f"Found {len(all_urls)} tracks to download.")
    print("URLs to process (from earliest to latest):")
    for date, url in unique_dated_urls:
        print(f"- {date.strftime('%Y-%m-%d')}: {url}")

    print(f"\nStarting download of {len(all_urls)} SoundCloud audios to: {output_dir}")
    downloaded_files = []
    failed_urls = []
    
    for url in all_urls:
        try:
            response = requests.head(url)
            if response.status_code == 404:
                print(f"Skipping URL with 404 status: {url}")
                continue
        except Exception as e:
            print(f"Error checking URL {url}: {str(e)}")
            continue
            
        result = download_soundcloud_audio(url, output_dir)
        if result:
            downloaded_files.append(result)
        else:
            failed_urls.append(url)
            
        time.sleep(2)

    print("\nSummary:")
    print(f"Total URLs processed: {len(all_urls)}")
    print(f"Successfully downloaded: {len(downloaded_files)}")
    print(f"Failed downloads: {len(failed_urls)}")
    
    if downloaded_files:
        print("\nDownloaded files:")
        for file in downloaded_files:
            print(f"- {file}")
            
    if failed_urls:
        print("\nFailed URLs:")
        for url in failed_urls:
            print(f"- {url}")

    if save_to_drive and downloaded_files:
        drive_path = "/content/drive/MyDrive/" + output_dir
        print(f"\nSaving files to Google Drive at: {drive_path}")

        if not os.path.exists(drive_path):
            os.makedirs(drive_path)

        os.system(f"cp -r {output_dir}/*.mp3 {drive_path}/")
        print(f"Files successfully saved to Google Drive")

    return downloaded_files, failed_urls

def retry_failed_downloads(failed_urls, output_dir, save_to_drive=False, max_retries=5):
    """
    Retry downloading failed URLs with protection against infinite loops.
    
    Args:
        failed_urls: List of URLs that failed to download in the first attempt
        output_dir: Directory to save downloaded files
        save_to_drive: Whether to save files to Google Drive
        max_retries: Maximum number of retry attempts per URL
        
    Returns:
        Tuple of (downloaded_files, still_failed_urls)
    """
    if not failed_urls:
        print("No failed downloads to retry.")
        return [], []
        
    print(f"Retrying {len(failed_urls)} failed downloads...")
    
    downloaded_files = []
    still_failed = []
    
    # Track retry attempts for each URL to prevent infinite loops
    retry_counts = {}
    max_retry_attempts = 1  # Only try once more to avoid excessive retries
    
    for url in failed_urls:
        # Check if we've already tried this URL too many times
        retry_counts[url] = retry_counts.get(url, 0) + 1
        
        if retry_counts[url] > max_retry_attempts:
            print(f"Skipping {url} - exceeded maximum retry attempts")
            still_failed.append(url)
            continue
            
        print(f"Retrying: {url} (Attempt {retry_counts[url]}/{max_retry_attempts})")
        result = download_soundcloud_audio(url, output_dir, max_retries=max_retries)
        
        if result:
            downloaded_files.append(result)
            print(f"Successfully downloaded on retry: {url}")
        else:
            still_failed.append(url)
            print(f"Failed again: {url}")
            
        time.sleep(5)
        
    print("\nRetry Summary:")
    print(f"Successfully downloaded: {len(downloaded_files)}")
    print(f"Still failed: {len(still_failed)}")
    
    if downloaded_files and save_to_drive:
        drive_path = "/content/drive/MyDrive/" + output_dir
        print(f"\nSaving files to Google Drive at: {drive_path}")
        
        if not os.path.exists(drive_path):
            os.makedirs(drive_path)
            
        os.system(f"cp -r {output_dir}/*.mp3 {drive_path}/")
        print("Files successfully saved to Google Drive")
        
    return downloaded_files, still_failed

In [4]:
# Example usage
profile_url = "https://soundcloud.com/radio-ergo"
start_date = "2024-07-01"
end_date = "2024-09-30"
output_dir = "data/soundcloud_2024-07-01_to_2024-09-30"
save_to_drive = False
max_retries=1

# Download files
downloaded_files, failed_urls = download_by_date_range(
    profile_url, 
    start_date, 
    end_date, 
    output_dir=output_dir,
    save_to_drive=save_to_drive
)

# Retry failed downloads if any
if failed_urls:
    print("\nRetrying failed downloads...")
    retry_downloaded, still_failed = retry_failed_downloads(
        failed_urls, 
        output_dir, 
        save_to_drive=save_to_drive,
        max_retries=max_retries
    )

Searching for tracks from 2024-07-01 to 2024-09-30...


URL does not match date pattern: https://soundcloud.com/radio-ergo/likes
URL does not match date pattern: https://soundcloud.com/radio-ergo/sets
URL does not match date pattern: https://soundcloud.com/radio-ergo/tracks
URL does not match date pattern: https://soundcloud.com/radio-ergo/comments
Manually found URL: https://soundcloud.com/radio-ergo/idaacadda-01-jul-2024 for date 2024-07-01
Manually found URL: https://soundcloud.com/radio-ergo/idaacadda-02-jul-2024 for date 2024-07-02
Manually found URL: https://soundcloud.com/radio-ergo/idaacadda-03-jul-2024 for date 2024-07-03
Manually found URL: https://soundcloud.com/radio-ergo/idaacadda-04-jul-2024 for date 2024-07-04
Manually found URL: https://soundcloud.com/radio-ergo/idaacadda-05-jul-2024 for date 2024-07-05
Manually found URL: https://soundcloud.com/radio-ergo/idaacadda-06-jul-2024 for date 2024-07-06
Manually found URL: https://soundcloud.com/radio-ergo/idaacadda-07-jul-2024 for date 2024-07-07
Manually found URL: https://sound

ERROR: [soundcloud] Unable to download JSON metadata: HTTP Error 404: Not Found (caused by <HTTPError 404: Not Found>)


Error downloading https://soundcloud.com/radio-ergo/idaacadda-04-jul-2024 (Attempt 1/3): ERROR: [soundcloud] Unable to download JSON metadata: HTTP Error 404: Not Found (caused by <HTTPError 404: Not Found>)
[soundcloud] Extracting URL: https://soundcloud.com/radio-ergo/idaacadda-04-jul-2024
[soundcloud] radio-ergo/idaacadda-04-jul-2024: Downloading info JSON


ERROR: [soundcloud] Unable to download JSON metadata: HTTP Error 404: Not Found (caused by <HTTPError 404: Not Found>)


...

Error downloading https://soundcloud.com/radio-ergo/idaacadda-06-sep-2024 (Attempt 1/1): ERROR: [soundcloud] Unable to download JSON metadata: HTTP Error 404: Not Found (caused by <HTTPError 404: Not Found>)
Failed to download https://soundcloud.com/radio-ergo/idaacadda-06-sep-2024 after 1 attempts
Failed again: https://soundcloud.com/radio-ergo/idaacadda-06-sep-2024
Retrying: https://soundcloud.com/radio-ergo/idaacadda-07-sep-2024 (Attempt 1/1)
[soundcloud] Extracting URL: https://soundcloud.com/radio-ergo/idaacadda-07-sep-2024
[soundcloud] radio-ergo/idaacadda-07-sep-2024: Downloading info JSON


ERROR: [soundcloud] Unable to download JSON metadata: HTTP Error 404: Not Found (caused by <HTTPError 404: Not Found>)


Error downloading https://soundcloud.com/radio-ergo/idaacadda-07-sep-2024 (Attempt 1/1): ERROR: [soundcloud] Unable to download JSON metadata: HTTP Error 404: Not Found (caused by <HTTPError 404: Not Found>)
Failed to download https://soundcloud.com/radio-ergo/idaacadda-07-sep-2024 after 1 attempts
Failed again: https://soundcloud.com/radio-ergo/idaacadda-07-sep-2024
Retrying: https://soundcloud.com/radio-ergo/idaacadda-30-sep-2024 (Attempt 1/1)
[soundcloud] Extracting URL: https://soundcloud.com/radio-ergo/idaacadda-30-sep-2024
[soundcloud] radio-ergo/idaacadda-30-sep-2024: Downloading info JSON


ERROR: [soundcloud] Unable to download JSON metadata: HTTP Error 404: Not Found (caused by <HTTPError 404: Not Found>)


Error downloading https://soundcloud.com/radio-ergo/idaacadda-30-sep-2024 (Attempt 1/1): ERROR: [soundcloud] Unable to download JSON metadata: HTTP Error 404: Not Found (caused by <HTTPError 404: Not Found>)
Failed to download https://soundcloud.com/radio-ergo/idaacadda-30-sep-2024 after 1 attempts
Failed again: https://soundcloud.com/radio-ergo/idaacadda-30-sep-2024

Retry Summary:
Successfully downloaded: 0
Still failed: 15
